<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ConceptNet_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pickle
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import requests
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [36]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [37]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [38]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [44]:
def ConceptNet_Sentences(data,labels):
  conceptnet_data=[]
  conceptnet_labels=[]
  try:
    for sentences in range(0,len(data)):
      current_sentence=data[sentences]
      conceptnet_data.append(current_sentence)
      conceptnet_labels.append(labels[sentences])
      cleaned_sentences=data[sentences].translate(str.maketrans('', '', string.punctuation))
      tokens_with_sw=nltk.word_tokenize(cleaned_sentences)
      tokens= [word for word in tokens_with_sw if not word in stopwords.words()]
      for i in range(1,3):
        temp=current_sentence
        for j in tokens:
          obj = requests.get('http://api.conceptnet.io/related/c/en/' + j + '?filter=/c/en').json()
          try:
            response=obj['related'][i]['@id']
          except Exception:
            continue
          response=re.sub(r'[^\w]', ' ', response)
          response=response[6:]
          temp=temp.replace(j,response)
        conceptnet_data.append(temp)
        conceptnet_labels.append(labels[sentences])
    except JSONDecodeError:
      pass
  return conceptnet_data,conceptnet_labels

SyntaxError: ignored

In [41]:
new_data,new_label = ConceptNet_Sentences(processed_data,processed_label)

JSONDecodeError: ignored

In [ ]:
import pickle
with open('processed_data.pkl', 'wb') as f:
  pickle.dump(new_data, f)
with open('processed_label.pkl', 'wb') as f:
  pickle.dump(new_label, f)